In [ ]:
#this is the second attempt at dynamic price modeling using ml techniques
#kaggle comp link: https://www.kaggle.com/competitions/home-data-for-ml-course/overview

In [48]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
#from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNet, ElasticNetCV

## Read data

In [38]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


## Target and validation

In [39]:

X = train.drop(['SalePrice'], axis=1)
y = np.log(train['SalePrice'])
y_binned = pd.qcut(y, q=50)
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.3, 
                                                    random_state=617,
                                                    stratify=y_binned)

## Col identification

In [40]:
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns
numeric_features

Index(['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd',
       'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF',
       'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea',
       'MiscVal', 'MoSold', 'YrSold'],
      dtype='object')

## Preprocessing

In [41]:
#numeric transformer

In [42]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy = 'mean')),
    ('scaler', StandardScaler())
])

In [43]:
#categorical transformer

In [44]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy ='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


In [45]:
#preprocessing pipeline

In [46]:
preprocessor = ColumnTransformer(transformers=[
    ('numeric', numeric_transformer, numeric_features),
    ('categorical', categorical_transformer, categorical_features)
])

In [49]:
model = ElasticNetCV(l1_ratio=1,
                     cv=5,
                     random_state=1031)

In [50]:
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', model)
]
                   
                   )

In [51]:
pipeline.fit(X_train, y_train)

,steps,"[('preprocessor', ...), ('regressor', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('numeric', ...), ('categorical', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [52]:
test_preds = np.exp(pipeline.predict(test))

In [53]:
submission = pd.DataFrame({
    'Id': test['Id'],
    'SalePrice': test_preds
})

In [54]:
submission

,Id,SalePrice
0,1461,122790.497831
1,1462,146796.505302
2,1463,173139.566943
3,1464,192931.990816
4,1465,196068.138689
...,...,...
1454,2915,81222.742348
1455,2916,79920.774663
1456,2917,165260.593626
1457,2918,117843.400304


In [55]:
val_preds = pipeline.predict(X_test)
rmse = mean_squared_error(
    np.exp(y_test),
    np.exp(val_preds)
)

print("Validation RMSE:", rmse)

Validation RMSE: 401463099.83256286


In [56]:
submission = pd.DataFrame({
    'Id': test['Id'],
    'SalePrice': test_preds
})

submission.to_csv('./submission.csv', index=False)

print("Submission file created successfully!")

Submission file created successfully!
